## Building Sub Graphes
Now that the graphes are computed we now need to extract the subgraphes based on the longest paths having the shortest distance  
iteratively removing them from the base graph until all the subgraphes have been extracted, this should be our devices.

In [6]:
import networkx as nx
import math
import pandas as pd
import numpy as np
from uuid import UUID
import operator
import multiprocessing

from itertools import chain
from itertools import product
from itertools import starmap
from functools import partial
import itertools

import sys  
sys.path.insert(0, './Functions/')

from Object import Position_graph
import Graph

In [7]:
def search_uuid_in_slices(obj, node):
    distance_slice_keys = list(obj.active_distance_slice.keys())
    for key in distance_slice_keys:
        if len(obj.active_distance_slice[key][obj.active_distance_slice[key]['device_mac']==node])>0:
            return key
    return None

def set_nodes_time_window_attribute(obj, nodes_list):
    nx.set_node_attributes(mygraph.graph, None, "time_window")
    for node in nodes_list:
        node_time_window = search_uuid_in_slices(obj, node)
        obj.graph.nodes[node]['time_window'] = node_time_window[0]
    return None

def generate_topological_order(obj):
    obj.topological_order = nx.topological_sort(nx.line_graph(obj.graph))
    
def test_and_keep_leaf(path_dict, leaves):
    '''
    Test if a path actually ends with a leaf, only keep these ones, delete the rest
    '''
    single_source_keys = list(path_dict.keys())
    for key in single_source_keys:
        if not (path_dict[key][-1] in leaves):
            del path_dict[key]
    return path_dict

def generate_path_from_roots_to_leaves(graph, roots, leaves):
    '''
    Generate all path from the graph that actually span from root to leaf.
    '''
    final_dict = {}
    total_paths = 0
    for root in roots:
        path_dict = dict(nx.single_source_bellman_ford_path(graph, source=root, weight='distance'))
        path_computed = test_and_keep_leaf(path_dict, leaves)
        final_dict[root] = path_computed
        total_paths = total_paths + len(list(path_computed.keys()))
    print(f"Computed {len(list(final_dict.keys()))} roots with a total of {total_paths} total paths!")
    return final_dict

def compute_distance_for_paths(graph, all_paths_list):
    distance_path_list = []
    for path in all_paths_list:
        pathGraph = nx.path_graph(path)
        distance_path_list.append((path, average_distance_per_path(graph, pathGraph)))
    return distance_path_list
        
def average_distance_per_path(graph, path):
    sum_distance = 0
    length_path = len(path)
    for ea in path.edges():
        #print from_node, to_node, edge's attributes
        sum_distance = sum_distance + graph.edges[ea[0], ea[1]]['distance']
    return (sum_distance, length_path)

def unpack_items(items):
    return [*items]

def remove_nodes_from_graph(graph, nodes):
    graph.remove_nodes_from(nodes)

def is_path(graph, path):
    try:
        return nx.is_simple_path(graph, path) 
    except:
        return False
    
def remove_devices_from_graph(graph, pourc=0.5):
    '''
    Compute all the paths from root to leaves and remove the lowest n from the graph based on % of the paths available.
    This function modifies the graph and returns a list of resulting sub graphs
    '''
    devices = []
    roots = [v for v, d in graph.in_degree() if d == 0]
    leaves = [v for v, d in graph.out_degree() if d == 0]

    all_paths_root_to_leaf = generate_path_from_roots_to_leaves(graph, roots, leaves)
    #unpacking the structure generated from the previous function call
    all_paths_list_root_to_leaf = Graph.put_in_list(all_paths_root_to_leaf)
    list_root_to_leaf_unchained = list(chain.from_iterable([unpack_items(d.values()) for d in all_paths_list_root_to_leaf]))
    #calculating the distance in each path to minimize it later
    root_to_leaf_paths_distance = compute_distance_for_paths(graph, list_root_to_leaf_unchained)
    #sorting by distance accrued
    root_to_leaf_paths_distance.sort(key=lambda x: x[1][0])
    
    
    for path in root_to_leaf_paths_distance[0:int(len(root_to_leaf_paths_distance)*pourc)]:
        if is_path(graph, path[0]):
            if(len(path[0])>2):
                devices.append([graph.subgraph(path[0]).copy(),path[1]])
            remove_nodes_from_graph(graph, path[0])
    
    print(f'    Found {len(devices)} device!')
            
    return devices

In [8]:
%%bigquery df_onroute_position_window
Select *
from data-prod-270222.datascience.position_raw

Downloading: 100%|██████████| 82228/82228 [00:00<00:00, 144645.85rows/s]


In [19]:
mygraph = Position_graph(df_onroute_position_window)
mygraph.window_kwargs = {
            'min_window_len':10,
            'min_value_len':5,
        }
mygraph.compute_window()
mygraph.compute_graph(seq_pourc = 0.25, n_windows = 3)
mygraph.compute_subgraphs(pourcentage_complete_paths = 0.5)

New empty graph
Length of dataset: 82228
Slice from 0 to 128 computed for distance stored under the active_distance_slice attribute


100%|██████████| 128/128 [00:53<00:00,  2.37it/s]


Computed 262 roots with a total of 3800 total paths!
    Found 76 device!
Added 76 devices subgraphs under the subgraph attribute


In [22]:
mygraph.subgraph.sort(key=lambda x: x[1][0])

In [23]:
mygraph.subgraph

[[<networkx.classes.digraph.DiGraph at 0x7f2c2b7ea490>, (0.0, 3)],
 [<networkx.classes.digraph.DiGraph at 0x7f2c2b7eaad0>, (0.0, 5)],
  (0.8395671428957812, 4)],
  (3.3306794232149137, 3)],
  (3.949040785154213, 7)],
  (4.704709734393525, 3)],
  (4.7896090042856265, 3)],
  (5.36977403420032, 51)],
  (6.645579136382999, 8)],
  (6.932268922060942, 3)],
  (8.537467858360522, 3)],
  (8.610815026218676, 53)],
  (9.479477686315192, 4)],
  (10.384577113562376, 3)],
  (10.74038917969785, 3)],
  (11.946559648859216, 4)],
  (12.600176777536632, 3)],
  (13.249105286269957, 4)],
  (13.627668255009944, 3)],
  (13.629563851064034, 5)],
  (15.23052289895405, 5)],
  (16.358384326353367, 4)],
  (16.732576901763395, 4)],
  (17.417104781345802, 3)],
  (17.84722892024999, 4)],
  (17.96720285495752, 3)],
  (18.034616159198332, 3)],
  (18.659112100916154, 3)],
  (19.105634730403608, 3)],
  (19.66905942469937, 4)],
  (21.065066266430023, 3)],
  (22.068996099157623, 5)],
  (22.186500563445236, 10)],
  (23.585